In [37]:
# This script is used to generate a table with data on ECOD domains. 
# Based on the input data, which is 5 files describing domains and genomes from the Refseq database in various ways, 
# we finally get a table in which each row describes a single domain in a specific genome.

In [1]:
# all the necessary imports to make the script work

import pandas as pd
import numpy as np
import subprocess
from Bio import Entrez, SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation


In [2]:
# thresholds against which domains that do not meet the specified assumptions will be filtered out

MINIMUM_QCOV_FOR_DOMAIN = 0
MINIMUM_SCOV_FOR_DOMAIN = 0.7
MINIMUM_PROB_FOR_DOMAIN = 90
MAXIMUM_EVAL_FOR_DOMAIN = 10**(-2)

In [3]:
# Paths to the input files, depending on where the file is the paths should be adjusted. The paths shown here are that of the project author. 

# The ECOD_DOMAIN_DESCRIPTION_FILEPATH variable describes the exact ECOD domain architecture with all its names, hierarchy levels and identifiers. 
# This is a file that can be directly downloaded from: http://prodata.swmed.edu/ecod/complete/distribution
ECOD_DOMAIN_DESCRIPTION_FILEPATH = "C:/Users/maksn/Desktop/magi_maks/input_files/ecod.develop283.domains.txt"



# The ECOD_DOMAIN_HITS_PATH variable specifies the path to a file with the exact results from HHblits, that is, with the results of how many domains we find for the given reprseq. 
# Such a file contains information about the identifier of the found domain, its probability, its names from T and F levels.
ECOD_DOMAIN_HITS_PATH = "C:/Users/maksn/Desktop/magi_maks/input_files/hhblits-ecod.txt"


# The variable REFSEQ_PROTEIN_NAME_TABLE specifies the path to the file in which there is a table with a description from which 
# genome and from which part of it we take reprseq (representative protein), based on these proteins we predict the occurring ECOD domains.
REFSEQ_PROTEIN_NAME_TABLE = "C:/Users/maksn/Desktop/magi_maks/input_files/name-table-full.txt"

In [4]:
# Auxiliary functions to process files


# Function rounds and changes to numeric some columns from processed dataframe

def parse_hhr_raw_result(hhblits_table):
    hhblits_table['qstart'] = pd.to_numeric(hhblits_table['qstart'])
    hhblits_table['qend'] = pd.to_numeric(hhblits_table['qend'])
    hhblits_table['sstart'] = pd.to_numeric(hhblits_table['sstart'])
    hhblits_table['send'] = pd.to_numeric(hhblits_table['send'])
    hhblits_table['prob'] = pd.to_numeric(hhblits_table['prob'])
    hhblits_table['eval'] = pd.to_numeric(hhblits_table['eval'])
    hhblits_table['qcov'] = round((hhblits_table['qend'] - hhblits_table['qstart'] + 1) / hhblits_table['qlength'], 4)
    hhblits_table['scov'] = round((hhblits_table['send'] - hhblits_table['sstart'] + 1) / hhblits_table['slength'], 4)
    hhblits_table['annotation'] = hhblits_table['annot'].str.lower()
    hhblits_table['hit_length'] = hhblits_table['qend'] - hhblits_table['qstart'] + 1
    return hhblits_table



# Function gets the uid from subject of ECOD domains hhblits subject

def get_uid_from_ecod_subject_id(subject_id):
    uid = subject_id.split("_")[1]
    return uid


# Function gets the identifier of domain from ECOD domains hhblits subject

def get_ecod_domain_id_from_ecod_subject_id(subject_id):
    ecod_domain_id = subject_id.split("_")[2]
    return ecod_domain_id



# This function is designed to integrate and consolidate annotations from ecod_domain_descriptions with a list of 
# distinct domains, facilitating further analysis and understanding of domain-specific information

def get_ecod_annotation_mapping(distinct_domains, ecod_domain_descriptions):
    unclassified_f_names = ["F_UNCLASSIFIED", "UNK_F_TYPE"]
    
    # Splitting f_id column into separate columns
    ecod_domain_descriptions[['x_ind', 'h_ind', 't_ind', 'f_ind']] = ecod_domain_descriptions['f_id'].str.split('.', expand=True)

    # Creating f_unclassified column
    ecod_domain_descriptions['f_unclassified'] = ecod_domain_descriptions['f_ind'].isna()

    # Replacing NA with empty string in f_ind column
    ecod_domain_descriptions['f_ind'] = ecod_domain_descriptions['f_ind'].fillna('NA')

    # Renaming f_id column to f_id_orig
    ecod_domain_descriptions.rename(columns={'f_id': 'f_id_orig'}, inplace=True)

    # Creating additional columns for hierarchical identifiers
    ecod_domain_descriptions['x_id'] = ecod_domain_descriptions['x_ind']
    ecod_domain_descriptions['h_id'] = ecod_domain_descriptions['x_ind'] + '.' + ecod_domain_descriptions['h_ind']
    ecod_domain_descriptions['t_id'] = ecod_domain_descriptions['x_ind'] + '.' + ecod_domain_descriptions['h_ind'] + '.' + ecod_domain_descriptions['t_ind']
    ecod_domain_descriptions['f_id'] = ecod_domain_descriptions['x_ind'] + '.' + ecod_domain_descriptions['h_ind'] + '.' + ecod_domain_descriptions['t_ind'] + '.' + ecod_domain_descriptions['f_ind']
    
    # Creating ecod.annotation.map DataFrame
    ecod_annotation_map = pd.DataFrame({'domain': distinct_domains})
    ecod_annotation_map['#uid'] = ecod_annotation_map['domain'].apply(lambda x: get_uid_from_ecod_subject_id(x)).astype(np.int64)
    ecod_annotation_map['ecod_domain_id'] = ecod_annotation_map['domain'].apply(lambda x: get_ecod_domain_id_from_ecod_subject_id(x))
    
    # Joining DataFrames
    ecod_annotation_map = ecod_annotation_map.merge(ecod_domain_descriptions, left_on=['#uid', 'ecod_domain_id'], right_on=['#uid', 'ecod_domain_id'], how='left')
    
    return ecod_annotation_map

In [5]:
# Read and process ECOD hits
ecod_domain_descriptions = pd.read_csv(ECOD_DOMAIN_DESCRIPTION_FILEPATH, skiprows=4, sep='\t')
ecod_domain_descriptions = ecod_domain_descriptions[['#uid', 'ecod_domain_id', 'arch_name', 'x_name', 'h_name', 't_name', 'f_name', 'f_id']]

In [6]:
# Load ECOD domain hits data from a CSV file into a DataFrame.
ecod_domain_hits_all = pd.read_csv(ECOD_DOMAIN_HITS_PATH, sep=',')

# Parse and process the raw HHR results in the DataFrame.
ecod_domain_hits_all = parse_hhr_raw_result(ecod_domain_hits_all)

# Filter the DataFrame to include only domain hits that meet specified criteria:
# - Maximum E-value threshold for domain hits.
# - Minimum query coverage (qcov) threshold for domain hits.
# - Minimum subject coverage (scov) threshold for domain hits.
# - Minimum probability (prob) threshold for domain hits.
ecod_domain_hits_all = ecod_domain_hits_all[
    (ecod_domain_hits_all['eval'] <= MAXIMUM_EVAL_FOR_DOMAIN) & 
    (ecod_domain_hits_all['qcov'] >= MINIMUM_QCOV_FOR_DOMAIN) & 
    (ecod_domain_hits_all['scov'] >= MINIMUM_SCOV_FOR_DOMAIN) & 
    (ecod_domain_hits_all['prob'] >= MINIMUM_PROB_FOR_DOMAIN)
    ]

# Rename the 'sname' column to 'domain' for clarity and consistency.
ecod_domain_hits_all = ecod_domain_hits_all.rename(columns={'sname': 'domain'})

In [8]:
# Retrieve unique domain names from the 'domain' column in ecod_domain_hits_all DataFrame.
distinct_domains = ecod_domain_hits_all['domain'].unique()

# Generate an annotation map using distinct_domains and ecod_domain_descriptions DataFrame.
ecod_annotation_map = get_ecod_annotation_mapping(distinct_domains=distinct_domains, ecod_domain_descriptions=ecod_domain_descriptions)

# Select specific columns of interest from ecod_annotation_map DataFrame.
ecod_annotation_map = ecod_annotation_map[['domain', 'h_id', 't_id', 'f_id', 'x_name', 'h_name', 't_name', 'f_name']]

In [9]:
# Merge ecod_domain_hits_all DataFrame with ecod_annotation_map DataFrame based on 'domain' column.
ecod_domains_hits = pd.merge(ecod_domain_hits_all, ecod_annotation_map, on='domain', how='left')

In [11]:
# This merge operation allows for integrating protein name information from prot_names into ecod_domains_hits, 
# facilitating further analysis or annotation of domain hits with associated protein names.


# Read the protein names table from REFSEQ_PROTEIN_NAME_TABLE into prot_names DataFrame.
prot_names = pd.read_csv(REFSEQ_PROTEIN_NAME_TABLE)

# Merge ecod_domains_hits DataFrame with prot_names DataFrame using 'qname' from ecod_domains_hits
# and 'repr.name' from prot_names, performing a left join.
ecod_domains_hits_to_orfs = pd.merge(ecod_domains_hits, prot_names, left_on='qname', right_on='repr.name', how='left')
ecod_domains_hits_to_orfs.drop(columns=['repr.name'], inplace=True)

In [14]:
# This code snippet effectively splits the 'ncbi.id' column into two separate columns ('genome_id' and 'gene_position'), 
# renames them appropriately, and then merges them back into the original DataFrame

# Split the 'ncbi.id' column in ecod_domains_hits_to_orfs DataFrame by ':' and expand into separate columns.
ecod_selected = ecod_domains_hits_to_orfs['ncbi.id'].str.split(':', expand=True)

# Rename the newly created columns.
ecod_selected.columns = ['genome_id', 'gene_position']

# Concatenate the new columns with the original ecod_domains_hits_to_orfs DataFrame along axis 1 (columns).
ecod_selected = pd.concat([ecod_domains_hits_to_orfs, ecod_selected], axis=1)

In [15]:
# This code segment adds a new column 'strand' to the ecod_selected DataFrame, indicating the strand direction (1 for positive strand, -1 for 
# negative strand, and None for unspecified cases) based on the format of 'gene_position' entries.

# Function definition to determine strand direction based on the entry format
def get_strand(entry):
    if entry.endswith('(+)'):
        return 1
    elif entry.endswith('(-)'):
        return -1
    else:
        return None 

# Apply get_strand function to create a new 'strand' column based on 'gene_position' column
ecod_selected['strand'] = ecod_selected['gene_position'].apply(lambda x: get_strand(x))

In [19]:
# This code snippet prepares and cleans the data in ecod_selected DataFrame, extracting start and end positions for domains, converting 
# columns to numeric types, and creating a unique identifier 'ncbi_id' for each record.

# Remove the last 3 characters from 'gene_position' column and assign to 'gene_position_clean'
ecod_selected['gene_position_clean'] = ecod_selected['gene_position'].str[:-3]

# Split 'gene_position_clean' column by '-' delimiter and expand into separate 'gene_start' and 'gene_end' columns
ecod_selected[['gene_start', 'gene_end']] = ecod_selected['gene_position_clean'].str.split('-', expand=True)

# Convert 'gene_start' and 'gene_end' columns to numeric type, coercing errors to NaN
ecod_selected['gene_start'] = pd.to_numeric(ecod_selected['gene_start'], errors='coerce')
ecod_selected['gene_end'] = pd.to_numeric(ecod_selected['gene_end'], errors='coerce')

# Calculate 'domain_start' and 'domain_end' based on 'gene_start', 'qstart', 'qend' columns
ecod_selected['domain_start'] = ecod_selected['gene_start'] + (ecod_selected['qstart'] - 1) * 3
ecod_selected['domain_end'] = ecod_selected['domain_start'] + (ecod_selected['qend'] - 1) * 3

# Concatenate 'genome_id' and 'gene_position' columns with ':' separator into 'ncbi_id' column
ecod_selected['ncbi_id'] = ecod_selected['genome_id'] +":"+ ecod_selected['gene_position']

In [22]:
# The text file surely.annot.protein.txt contains data on the mapping of representative proteins to their functions, extracted from 
# the PHROGs database. Specifically, this file contains two columns - the identifier of the protein and the identifier of the 
# function that has been predicted for the protein.
anotacja_funkcjonalna_repr = pd.read_csv('C:/Users/maksn/Downloads/surely.annot.proteins.txt')
anotacja_funkcjonalna_repr = anotacja_funkcjonalna_repr[["qname", "annotation.index"]]

# The text file annotation.indices.txt contains data on the mapping of representative proteins to their functions, taken from the 
# PHROGs database. Specifically, this file contains three columns - function identifier, function name and function category.
anotacja_funkcjonalna_ind = pd.read_csv('C:/Users/maksn/Downloads/annotation.indices.txt')
anotacja_funkcjonalna_ind = anotacja_funkcjonalna_ind.rename(columns={'annotation': 'func_annot', 'category': 'func_category'})

# Merge two DataFrames based on the index of function column using left join.
merged_functional = pd.merge(anotacja_funkcjonalna_repr, anotacja_funkcjonalna_ind, on='annotation.index', how='left')

# Merge 'ecod_selected' DataFrame with 'merged_functional' DataFrame based on 'qname' column using left join
ecod_selected = pd.merge(ecod_selected, merged_functional, on='qname', how='left')

In [32]:
# Select specific columns from 'ecod_selected' DataFrame to form 'ecod_final'
ecod_final = ecod_selected[['genome_id', 'gene_start', 'gene_end', 'domain_start', 'domain_end', 't_name', 'f_id', 'ncbi_id' , 'strand', 'func_annot', 'func_category']]

# Rename the 'ncbi_id' column to 'gene_id' in 'ecod_final' DataFrame
ecod_final = ecod_final.rename(columns={'ncbi_id':'gene_id'})

In [35]:
# Save finalised ECOD domains dataframe into csv file
ecod_final.to_csv('C:/Users/maksn/Downloads/ecod_final_to_clinker_annot.csv', index=False)